# Banco de dados

Banco de dados é uma estrutura organizada para armazenar, gerenciar e recuperar informações de forma eficiente. Um termo conhecido é o CRUD (create, read, update e delete). Conheceremos agora um banco de dados para testes, chamado Sqlite3.

In [1]:
# importando a biblioteca para fazer operações em banco de dados sqlite3
import sqlite3
# importando a bibliotecas pandas para transformar dados e tabelas do banco em data frames 
import pandas as pd

# CONNECTION

In [2]:
# criaremos a conexão com nosso banco de dados
# se ele não existir, será criado
conn = sqlite3.connect("database.db")

# CREATE TABLE

In [3]:
# Podemos agora criar tabelas
# criaremos a tabela (table) tasks com campos (fields) id, description e done
conn.execute("""
    CREATE TABLE IF NOT EXISTS tasks
    (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        description TEXT,
        done INTEGER
    )    
""")
# após executar uma operação é necessário salvá-la
conn.commit()

# CREATE

In [4]:
# criaremos agora um registro (row) na tabela tasks
# o registro terá valor 'Estudar seaborn' para description e 0 para done.
conn.execute("""
    INSERT INTO tasks (description, done)
    VALUES ('Estudar seaborn', 0);    
""")
# novamente, após executar o comando é necessário salvar
conn.commit()

IntegrityError: NOT NULL constraint failed: tasks.category_id

In [5]:
# TODO

# crie um registro (row) na tabela tasks
# o registro terá valor 'Estudar python' para description e 1 para done.
conn.execute("""
 
""")
# novamente, após executar o comando é necessário salvar
conn.commit()

In [6]:
# TODO

# crie um registro (row) na tabela tasks
# o registro terá valor 'Entregar livro' para description e 0 para done.
conn.execute("""
 
""")
# novamente, após executar o comando é necessário salvar
conn.commit()

# READ

In [7]:
# podemos agora visualizar todos (*) os registros da (FROM) tabela tasks
results = conn.execute("""
    SELECT *
    FROM tasks
""").fetchall()
# imprimindo cada resultado
for result in results:
    print(result)

(1, 'Assistir Netflix', 1, 2)
(2, 'Estudar matplotlib', 0, 3)
(3, 'Estudar seaborn', 0, 3)
(4, 'Estudar Pandas', 0, 3)
(5, 'Lavar louça', 0, 1)


In [8]:
# ver os valores de description da (FROM) tabela tasks onde (WHERE) done = 0
results = conn.execute("""
    SELECT description
    FROM tasks
    WHERE done=0
""").fetchall()
# imprimindo cada resultado
for result in results:
    print(result)

('Estudar matplotlib',)
('Estudar seaborn',)
('Estudar Pandas',)
('Lavar louça',)


In [9]:
# TODO

# ver os valores de description da (FROM) tabela tasks onde (WHERE) done = 0 e description começa com 'Estudar'
results = conn.execute("""

""").fetchall()
# imprimindo cada resultado
for result in results:
    print(result)

# UPDATE

In [10]:
# Atualizar (update) na tabela tasks as linhas (rows) cujo id = 1 fazendo (SET) done = 1
conn.execute("""
    UPDATE tasks
    SET done=1
    WHERE id=1
""")
# salvando as atualizações
conn.commit()

In [11]:
# TODO

# Atualizar (update) na tabela tasks as linhas (rows) cujo description = "Estudar seaborn" fazendo (SET) description = "Estudar matplotlib"
conn.execute("""

""")
# salvando as atualizações
conn.commit()

# DELETE

In [12]:
# deletar da (FROM) tabela tasks as linhas (rows) onde id = 1
conn.execute("""
    DELETE FROM tasks
    WHERE id=1
""")
conn.commit()

# Pandas query

Quando se fala em tabelas, uma das bibliotecas mais utilizar é a biblioteca pandas. Para nossa conveniência, o método read_sql_query da biblioteca retorna data frames de consultas SQL.

In [15]:
df = pd.read_sql_query(
    """
       SELECT *
       FROM tasks
    """,
    conn
)
df

,id,description,done,category_id
0,2,Estudar matplotlib,0,3
1,3,Estudar seaborn,0,3
2,4,Estudar Pandas,0,3
3,5,Lavar louça,0,1


# AGRUPAMENTO E AGREGAÇÃO

Existe um paralelo entre os métodos da biblioteca pandas e as queries SQL. Vejamos por exemplo como contar as linhas da tabela tasks onde done = 1

In [16]:
df['done'].value_counts()

done
0    4
Name: count, dtype: int64

Podemos fazer diretamente usando SQL:

In [17]:
results = conn.execute("""
    SELECT done, COUNT(*) AS done_tasks
    FROM tasks
    GROUP BY done;      
""").fetchall()

for result in results:
    print(result)

(0, 4)


# Encerrando a conexão

In [18]:
conn.close()

# RELACIONAMENTO

Antes de começar você pode apagar o arquivo database.db, vamos recriá-lo. Se decidir por não apagar, mude o nome do arquivo dessa nova conexão.

In [ ]:
conn = sqlite3.connect("database.db")

conn.execute("""
    CREATE TABLE categories 
    (
      id    INTEGER PRIMARY KEY, 
      name  TEXT NOT NULL
    );
""")

conn.execute(
    """
        INSERT INTO categories (name)
        VALUES
          ('casa'),
          ('laser'),
          ('faculdade')
    """
)
conn.commit()

In [20]:
conn.close()

A tabela tasks agora terá uma chave estrangeira (FOREIGN KEY) que faz referência ao campo id da tabela categories. Logo estas duas tabelas estão conectadas.

In [ ]:
conn.execute("""
    CREATE TABLE tasks 
    (
      id            INTEGER PRIMARY KEY AUTOINCREMENT, 
      description   TEXT NOT NULL,
      done          INTEGER,
      category_id   INTEGER NOT NULL,
      FOREIGN KEY (category_id) REFERENCES categories (id)
    )
""")
conn.commit()

<center><img src="./assets/tasks_categories_relationship.png" width="400px"></center>

# JOIN

Nosso próximo passo é fazer uma querie que junta as duas tabelas anteriores numa só. Antes, vamos inserir alguns dados na tabela tasks.

In [ ]:
# insert many
conn.execute(
    """
    INSERT INTO 'tasks' (description, done, category_id)
    VALUES
      ('Assistir Netflix', 1, 2),
      ('Estudar matplotlib', 0, 3),
      ('Estudar seaborn', 0, 3),
      ('Estudar Pandas', 0, 3),
      ('Lavar louça', 0, 1)
    """
)
conn.commit()

Agora sim, juntaremos as tabelas.

In [ ]:
tasks_with_categories = conn.execute(
    """
    SELECT t.id, t.description, t.done, c.name 
    FROM tasks AS t 
    INNER JOIN categories AS c
      ON c.id = t.category_id
    ORDER BY c.name
    """
).fetchall()

for task in tasks_with_categories:
    print(task)

Usando pandas para ter o resultado como um objeto data frame.

In [ ]:
df = pd.read_sql_query(
    """
    SELECT t.id, t.description, t.done, c.name 
    FROM tasks AS t 
    INNER JOIN categories AS c
      ON c.id = t.category_id
    ORDER BY c.name
    """,
    conn
)
df

# Close connection

In [ ]:
conn.close()

# Exercícios

<img src="assets/sqlite-sample-database-color.jpg">

Faça o download do bando SQLite3 no site: https://www.sqlitetutorial.net/sqlite-sample-database/

O arquivo também está na pastas assets, caso não queira fazer o download.

Usando os conhecimentos da aula, resolva os seguinte problemas:

1. Fornecer uma consulta que mostre os clientes (apenas seus nomes completos, ID do cliente e país) que não estão nos EUA.

In [ ]:
conn = sqlite3.connect("chinook.db")
df = pd.read_sql_query(
    """
        SELECT CustomerId, FirstName || ' ' || LastName AS FullName, Country
        FROM customers
        WHERE Country != 'USA'
    """,
    conn
)
conn.close()
df

2. Fornecer uma consulta que mostre apenas os clientes do Brasil.

In [ ]:
# TODO

3. Fornecer uma consulta que mostre as faturas dos clientes que são do Brasil. A tabela resultante deve mostrar o nome completo do cliente, ID da fatura, data da fatura e país de cobrança.

In [ ]:
# TODO

4. Fornecer uma consulta que mostre apenas os funcionários que são agentes de vendas.

In [ ]:
# TODO

5. Fornecer uma consulta que mostre uma lista única de países de cobrança da tabela de faturas.

In [ ]:
# TODO

6. Fornecer uma consulta que mostre as faturas associadas a cada agente de vendas. A tabela resultante deve incluir o nome completo do agente de vendas.

In [ ]:
# TODO

7. Fornecer uma consulta que mostre o valor total da fatura, nome do cliente, país e nome do agente de vendas de todas as faturas e clientes.

In [ ]:
# TODO

8. Quantas faturas houve em 2009 e 2011? Quais são as vendas totais respectivas para cada um desses anos?

In [ ]:
# TODO

9. Analisando a tabela InvoiceLine, forneça uma consulta que CONTE o número de itens para o ID da fatura 37.

In [ ]:
# TODO

10. Analisando a tabela invoice_items, forneça uma consulta que CONTE o número de itens para cada fatura. DICA: AGRUPAR POR

In [ ]:
# TODO

11. Fornecer uma consulta que inclua o nome da faixa para cada linha da tabela de itens da fatura.

In [ ]:
# TODO

12. Forneça uma consulta que mostre o nome da música e o artista para cada linha da tabela de itens da fatura.

In [ ]:
# todo

13. Fornecer uma consulta que mostre o número de faturas por país. DICA: AGRUPAR POR

In [ ]:
# TODO

14. Forneça uma consulta que mostra o número total de música por playlist. O nome das playlists devem aparecer na tabela resultante.

In [ ]:
# TODO

15.Forneça uma consulta que mostra todos os nomes das músicas, mas não mostra ids. A tabela resultante deve incluir o nome do álbum, tipo de media e gênero.

In [ ]:
# TODO

16. Forneça uma consulta que mostra todas as faturas por id e a contagem de itens por fatura.

In [ ]:
# TODO

17. Fornecer uma consulta que mostro o número total de vendas de cada vendedor.

In [ ]:
# TODO

18. Qual vendedor (sales agent) mais vendeu em 2009?

In [ ]:
# TODO

19. Qual vendedor (sales agent) mais vendeu em 2010?

In [ ]:
# TODO

20. Qual vendedor (sales agent) mais vendeu considerando toda base de dados?

In [ ]:
# TODO

21. Forneça uma consulta que mostra o número de clientes inscritos para cada vendedor (sales agente).

In [ ]:
# TODO

22. Forneça uma consulta que mostra o total de vendas por país.

In [ ]:
# TODO

23. De qual país os clientes mais gastaram?

In [ ]:
# TODO

24. Forneça uma consulta que mostra a faixa mais comprada de 2013.

In [ ]:
# TODO

25. Forneça uma consulta que mostra as top 5 faixas mais compradas de todos os tempos.

In [ ]:
# TODO

26. Forneça uma consulta que mostra os top 3 artistas mais vendidos de toda base de dados.

In [ ]:
# TODO

27. Forneça uma query que mostra qual o tipo de media mais comprado.

In [ ]:
# TODO

### Referência para os exercícios:

http://www.berkut.mk.ua/download/pdf/db/task_vesna.pdf